In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install setfit

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd, csv
#my_cols = ['label', 'tagged_in_context']
trainset= pd.read_csv('/content/drive/MyDrive/propaganda_train.tsv', sep='\t')
valset= pd.read_csv('/content/drive/MyDrive/propaganda_val.tsv',  sep="\t")

In [ ]:
# trainset['tagged_in_context']= trainset['tagged_in_context'].str.replace('<BOS>','[CLS]')
# trainset['tagged_in_context']= trainset['tagged_in_context'].str.replace('<EOS>','[SEP]')
# valset['tagged_in_context']= valset['tagged_in_context'].str.replace('<BOS>','[CLS]')
# valset['tagged_in_context']= valset['tagged_in_context'].str.replace('<EOS>','[SEP]')
# valset

In [ ]:
propaganda_set= trainset[trainset['label'] != 'not_propaganda'].copy()
not_propaganda_set = trainset[trainset['label'] == 'not_propaganda']
val_propaganda_set= valset[valset['label'] != 'not_propaganda'].copy()
val_not_propaganda_set = valset[valset['label'] == 'not_propaganda']

prop_labels = propaganda_set.loc[:,['label']]
for val in prop_labels:
  propaganda_set['label']= val.replace(val, 'propaganda')

val_prop_labels = val_propaganda_set['label']
for val in val_prop_labels:
  val_propaganda_set['label']= val.replace(val, 'propaganda')

propaganda_set
train_set = pd.concat([propaganda_set, not_propaganda_set], ignore_index= True)
val_set = pd.concat([val_propaganda_set, val_not_propaganda_set], ignore_index= True)

In [ ]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2414 entries, 0 to 2413
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   label              2414 non-null   object
 1   tagged_in_context  2414 non-null   object
dtypes: object(2)
memory usage: 37.8+ KB


In [ ]:
val_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   label              580 non-null    object
 1   tagged_in_context  580 non-null    object
dtypes: object(2)
memory usage: 9.2+ KB


## Fine Tuning Bert

Step 1:
We convert the label names to integers which will be used by BERT. The code below

In [ ]:
#first we need a map for the labels
#Here, we make a list of all of the unique labels in the training and testing dataframes, and sort the labels alphabetically
labellist=sorted(list(set(train_set['label'].unique()).union(set(val_set['label'].unique()))))

labels={label:i for i,label in enumerate(labellist)}
labels

{'not_propaganda': 0, 'propaganda': 1}

In [ ]:
#Creating a reverse of the labels dictionary
def reverse_idx(adict):
  reverse_idx= {}
  for key,value in adict.items():
    reverse_idx[value]= key
  return reverse_idx

reverse_index= reverse_idx(labels)
reverse_index

{0: 'not_propaganda', 1: 'propaganda'}

In [ ]:
#Using the BERT tokenizer to split the sentences into tokens
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self,df,column='tagged_in_context'):
        self.labels=[labels[label] for label in df['label']]
        self.texts=[tokenizer(text.lower(),padding='max_length',max_length=512,
                              truncation=True, return_tensors="pt") for text in df[column]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self,idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self,idx):
        return self.texts[idx]

    def __getitem__(self,idx):
        batch_texts=self.get_batch_texts(idx)
        batch_y=self.get_batch_labels(idx)

        return batch_texts,batch_y


train_data=Dataset(train_set)
test_data=Dataset(val_set)

In [ ]:
use_cuda=torch.cuda.is_available()
if use_cuda:
  print("GPU acceleration enabled")
else:
  print("GPU acceleration NOT enabled.  If using Colab, have you changed the runtype type and selected GPU as the hardware accelerator?")
device=torch.device("cuda" if use_cuda else "cpu")

GPU acceleration enabled


In [ ]:
def prepare_inputs(input1,label,device):
  label=label.to(device)
  mask=input1['attention_mask'].to(device)
  input_id=input1['input_ids'].squeeze(1).to(device)
  return (input_id,mask,label)

In [ ]:
#now we need to put a simple classification layer on top of BERT

from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self,dropout=0.5,num_classes=2):

        super(BertClassifier,self).__init__()

        self.bert=BertModel.from_pretrained('bert-base-uncased')
        self.dropout=nn.Dropout(dropout)
        self.linear=nn.Linear(768,num_classes)
        self.relu=nn.ReLU()

    def forward(self,input_id,mask):

        last_hidden_layer,pooled_output = self.bert(input_ids=input_id,attention_mask=mask,return_dict=False)
        dropout_output=self.dropout(pooled_output)
        linear_output=self.linear(dropout_output)
        final_layer=self.relu(linear_output)

        return final_layer

In [ ]:
#we now need a training loop

from torch.optim import Adam
from tqdm import tqdm  #useful library to report on progress through an iteration



def train(model, train_data,val_data,learning_rate,epochs):

    train_dataloader=torch.utils.data.DataLoader(train_data,batch_size=2,shuffle=True)
    val_dataloader=torch.utils.data.DataLoader(test_data,batch_size=2)

    use_cuda=torch.cuda.is_available()
    device=torch.device("cuda" if use_cuda else "cpu")

    criterion=nn.CrossEntropyLoss()
    optimizer=Adam(model.parameters(),lr=learning_rate)

    if use_cuda:
        model=model.cuda()
        criterion=criterion.cuda()

    for epoch_num in range(epochs):
        total_acc_train=0
        total_loss_train=0
        model.train()
        for train_input,train_label in tqdm(train_dataloader):

            input_id,mask, train_label=prepare_inputs(train_input,train_label,device)

            output=model(input_id,mask)

            batch_loss=criterion(output,train_label.long())
            total_loss_train +=batch_loss.item()

            acc=(output.argmax(dim=1)==train_label).sum().item()
            total_acc_train+=acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val=0
        total_loss_val=0
        model.eval()
        with torch.no_grad():
            for val_input,val_label in val_dataloader:

                input_id,mask, val_label=prepare_inputs(val_input,val_label,device)

                output=model(input_id,mask)

                batch_loss=criterion(output,val_label.long())

                total_loss_val+=batch_loss.item()

                acc=(output.argmax(dim=1)==val_label).sum().item()
                total_acc_val+=acc

        print(f'Epochs: {epoch_num+1} | Train Loss: {total_loss_train / len(train_data):.3f} | Train Accuracy: {total_acc_train/len(train_data):.3f}')
        print(f'Val loss: {total_loss_val/len(val_data):.3f} | Val Accuracy: {total_acc_val / len(val_data):.3f}')


In [ ]:
EPOCHS=5
model=BertClassifier(num_classes=len(labels.keys()))
LR=1e-6


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train(model,train_data,test_data,LR,EPOCHS)

100%|██████████| 1207/1207 [04:16<00:00,  4.71it/s]


Epochs: 1 | Train Loss: 0.345 | Train Accuracy: 0.530
Val loss: 0.340 | Val Accuracy: 0.636


100%|██████████| 1207/1207 [04:16<00:00,  4.70it/s]


In [ ]:
output_dir="bert-base-uncased-bookclassifier"
torch.save(model,output_dir)

In [ ]:
input_dir="bert-base-uncased-bookclassifier"
complete_model=torch.load(input_dir)

In [ ]:
batchsize=2
def evaluate(model,test_dataset):
    model.eval()
    test_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=batchsize)

    use_cuda=torch.cuda.is_available()
    device=torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model=model.cuda()

    total_acc_test=0
    with torch.no_grad():
        count=0
        predictions=[]
        for test_input,test_label in tqdm(test_dataloader):
            count+=batchsize
            test_label=test_label.to(device)
            mask=test_input['attention_mask'].to(device)
            input_id=test_input['input_ids'].squeeze(1).to(device)
            output=model(input_id,mask)
            #print(output.argmax(dim=1),test_label)
            predictions.append(output.argmax(dim=1))  #save the prediction for further analysis
            acc=(output.argmax(dim=1)==test_label).sum().item()

            total_acc_test+=acc
            if count%100==0:
                print(f'Accuracy so far = {total_acc_test/count: .3f}')

    print(f'Test accuracy: {total_acc_test/len(test_dataset): .3f}')
    return predictions

In [ ]:
predictions=evaluate(model, test_data)

In [ ]:
flattened=[]
for batch in predictions:
    for pred in batch:
        flattened.append(reverse_index[pred.item()])
val_set['prediction']=flattened
val_set.head(10)

In [ ]:
all_labels=val_set['label']
all_predictions=val_set['prediction']

In [ ]:
tp={}
fp={}
fn={}
tn={}

for label1,pred1 in zip(all_labels,all_predictions):
    for label in labels.keys():
        if label1==label:
            if pred1==label:
                tp[label]=tp.get(label,0)+1
            else:
                fn[label]=fn.get(label,0)+1

        else:
            if pred1==label:
                fp[label]=fp.get(label,0)+1
            else:
                tn[label]=tn.get(label,0)+1



precision={label:value/(value+fp.get(label,0)) for label,value in tp.items()}
recall={label:value/(value+fn.get(label,0)) for label,value in tp.items()}
f1={label:(2*value*recall.get(label,0))/(value+recall.get(label,0)) for label,value in precision.items()}


In [ ]:
recall

 # Classifying Propaganda Techniques using BERT

## Fine Tuning BERT

In [ ]:
propaganda_train_set= trainset[trainset['label'] != 'not_propaganda'].copy()
propaganda_val_set= valset[valset['label'] != 'not_propaganda'].copy()
propaganda_train_set= propaganda_train_set.reset_index(drop=True)
#propaganda_train_set= propaganda_train_set.drop(['index'], axis=1)
propaganda_val_set= propaganda_val_set.reset_index(drop=True)
#propaganda_val_set= propaganda_val_set.drop(['index'], axis=1)
propaganda_train_set.head(3)

,label,tagged_in_context
0,flag_waving,The Obama administration misled the <BOS> Amer...
1,loaded_language,"Hitler <BOS> annihilated <EOS> 400,000 Germans..."
2,doubt,"As noted above, at this point literally every ..."


In [ ]:
propaganda_train_set = propaganda_train_set.rename(columns={'tagged_in_context': 'text'})
propaganda_val_set = propaganda_val_set.rename(columns={'tagged_in_context': 'text'})

In [ ]:
propaganda_train_set.head(2)

,label,text
0,flag_waving,The Obama administration misled the <BOS> Amer...
1,loaded_language,"Hitler <BOS> annihilated <EOS> 400,000 Germans..."


In [ ]:
#first we need a map for the labels
#Here, we make a list of all of the unique labels in the training and testing dataframes, and sort the labels alphabetically
labellist=sorted(list(set(propaganda_train_set['label'].unique()).union(set(propaganda_val_set['label'].unique()))))
prop_labels={label:i for i,label in enumerate(labellist)}
prop_labels

{'appeal_to_fear_prejudice': 0,
 'causal_oversimplification': 1,
 'doubt': 2,
 'exaggeration,minimisation': 3,
 'flag_waving': 4,
 'loaded_language': 5,
 'name_calling,labeling': 6,
 'repetition': 7}

In [ ]:
#Creating a reverse of the labels dictionary
def reverse_idx(adict):
  reverse_idx= {}
  for key,value in adict.items():
    reverse_idx[value]= key
  return reverse_idx

reverse_index= reverse_idx(prop_labels)
reverse_index

{0: 'appeal_to_fear_prejudice',
 1: 'causal_oversimplification',
 2: 'doubt',
 3: 'exaggeration,minimisation',
 4: 'flag_waving',
 5: 'loaded_language',
 6: 'name_calling,labeling',
 7: 'repetition'}

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self,df,column='text'):
        self.labels=[prop_labels[label] for label in df['label']]
        self.texts=[tokenizer(text.lower(),padding='max_length',max_length=512,truncation=True,return_tensors="pt") for text in df[column]]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self,idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self,idx):
        return self.texts[idx]

    def __getitem__(self,idx):
        batch_texts=self.get_batch_texts(idx)
        batch_y=self.get_batch_labels(idx)

        return batch_texts,batch_y


prop_train_data=Dataset(propaganda_train_set)
prop_test_data=Dataset(propaganda_val_set)

In [ ]:
x_final = propaganda_train_set.drop('label',axis=1)
y_final= propaganda_train_set['label']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [ ]:
y_train= list(y_train)
y_test= list(y_test)

In [ ]:
#labels= y_train['label']
x_train['label']= y_train
x_test['label']= y_test
propaganda_train_set = x_train.copy()
propaganda_test_set = x_test.copy()

In [ ]:
propaganda_train_set= propaganda_train_set.reset_index(drop=True)
propaganda_test_set= propaganda_test_set.reset_index(drop=True)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(propaganda_train_set)
test_dataset = Dataset.from_pandas(propaganda_test_set)
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 404
})

In [ ]:
from setfit import SetFitModel, SetFitTrainer, sample_dataset
from sentence_transformers.losses import CosineSimilarityLoss

In [ ]:
# Load a SetFit model from Hub
from setfit import SetFitModel
num_classes = 8
model = SetFitModel.from_pretrained( "sentence-transformers/all-mpnet-base-v2",head_params = {
        "max_iter": 54,
        "solver": "lbfgs",
    })



# model = SetFitModel.from_pretrained(
#     multi_target_strategy="one-vs-rest"
#     use_differentiable_head=True,
#     head_params={"out_features": num_classes},)

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    num_epochs=1,
    num_iterations=40,
    batch_size=4,  # <- You can probably toy with this, e.g. put it higher for slightly higher training speed
    learning_rate=1.047e-05
    #column_mapping={"tagged_in_context": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.train()


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/40 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 65520
  Num epochs = 1
  Total optimization steps = 16380
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16380 [00:00<?, ?it/s]

In [ ]:
trainer.evaluate()

# Push model to the Hub
#trainer.push_to_hub("my-awesome-setfit-model")

# Download from Hub and run inference
#model = SetFitModel.from_pretrained("lewtun/my-awesome-setfit-model")
# Run inference
#preds = model(["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"])


***** Running evaluation *****


ValueError: ignored

In [ ]:
x_test= propaganda_val_set.text
y_test =  propaganda_val_set.label

In [ ]:
y_pred

In [ ]:
y_pred =  model.predict(x_test)
y_true = y_test
from sklearn import metrics
print(metrics.classification_report(y_pred, y_true))

# y_pred =   np.argmax(model.predict(x_val_pad), axis  =  1)
# y_true = np.argmax(y_val, axis = 1)
# from sklearn import metrics
# print(metrics.classification_report(y_pred, y_true))

                           precision    recall  f1-score   support

 appeal_to_fear_prejudice       0.70      0.77      0.73        39
causal_oversimplification       0.71      0.51      0.59        43
                    doubt       0.61      0.62      0.61        37
exaggeration,minimisation       0.71      0.57      0.63        35
              flag_waving       0.67      0.90      0.76        29
          loaded_language       0.65      0.71      0.68        34
    name_calling,labeling       0.65      0.80      0.71        25
               repetition       0.53      0.46      0.49        37

                 accuracy                           0.65       279
                macro avg       0.65      0.67      0.65       279
             weighted avg       0.65      0.65      0.65       279



In [ ]:
model.predict(x_test)

In [ ]:
#now we need to put a simple classification layer on top of BERT

from torch import nn
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

class BertClassifier(nn.Module):

    def __init__(self,dropout=0.5,num_classes=8):

        super(BertClassifier,self).__init__()

        self.bert=BertModel.from_pretrained('bert-base-uncased')
        self.dropout=nn.Dropout(dropout)
        self.linear=nn.Linear(768,num_classes)
        self.relu=nn.ReLU()

    def forward(self,input_id,mask):

        last_hidden_layer,pooled_output = self.bert(input_ids=input_id,attention_mask=mask,return_dict=False)
        dropout_output=self.dropout(pooled_output)
        linear_output=self.linear(dropout_output)
        final_layer=self.relu(linear_output)

        return final_layer



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class BertLstmClassifier(nn.Module):

    def __init__(self,dropout=0.5,num_classes=8):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.lstm = nn.LSTM(input_size= 768, hidden_size=768,
                            num_layers=2, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(768*2, num_classes)
        self.softmax = nn.Softmax()

    def forward(self, input_ids, mask):
        last_hidden_layer,pooled_output = self.bert(input_ids=input_ids, attention_mask=mask)
        lstm_output = self.lstm(pooled_output)
        #linear_output = self.linear(lstm_output)
        linear_output = self.linear(lstm_output[:, -1, :])
        final_layer = self.softmax(linear_output)
        return final_layer

In [ ]:
#we now need a training loop

from torch.optim import Adam
from tqdm import tqdm  #useful library to report on progress through an iteration



def train(model, train_data,val_data,learning_rate,epochs):

    train_dataloader=torch.utils.data.DataLoader(prop_train_data,batch_size=2,shuffle=True)
    val_dataloader=torch.utils.data.DataLoader(prop_test_data,batch_size=2)

    use_cuda=torch.cuda.is_available()
    device=torch.device("cuda" if use_cuda else "cpu")

    criterion=nn.CrossEntropyLoss()
    optimizer=Adam(model.parameters(),lr=learning_rate)

    if use_cuda:
        model=model.cuda()
        criterion=criterion.cuda()

    for epoch_num in range(epochs):
        total_acc_train=0
        total_loss_train=0
        model.train()
        for train_input,train_label in tqdm(train_dataloader):

            input_id,mask, train_label=prepare_inputs(train_input,train_label,device)

            output=model(input_id,mask)

            batch_loss=criterion(output,train_label.long())
            total_loss_train +=batch_loss.item()

            acc=(output.argmax(dim=1)==train_label).sum().item()
            total_acc_train+=acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val=0
        total_loss_val=0
        model.eval()
        with torch.no_grad():
            for val_input,val_label in val_dataloader:

                input_id,mask, val_label=prepare_inputs(val_input,val_label,device)

                output=model(input_id,mask)

                batch_loss=criterion(output,val_label.long())

                total_loss_val+=batch_loss.item()

                acc=(output.argmax(dim=1)==val_label).sum().item()
                total_acc_val+=acc

        print(f'Epochs: {epoch_num+1} | Train Loss: {total_loss_train / len(prop_train_data):.3f} | Train Accuracy: {total_acc_train/len(prop_train_data):.3f}')
        print(f'Val loss: {total_loss_val/len(prop_test_data):.3f} | Val Accuracy: {total_acc_val / len(prop_test_data):.3f}')


In [ ]:
EPOCHS=5
model=BertLstmClassifier(num_classes=len(prop_labels.keys()))
LR=1e-6


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train(model,prop_train_data,prop_test_data,LR,EPOCHS)

  0%|          | 0/612 [00:00<?, ?it/s]


AttributeError: ignored

In [ ]:
output_dir="bert-base-uncased-bookclassifier"
torch.save(model,output_dir)

In [ ]:
input_dir="bert-base-uncased-bookclassifier"
complete_model=torch.load(input_dir)

In [ ]:
batchsize=2
def evaluate(model,test_dataset):
    model.eval()
    test_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=batchsize)

    use_cuda=torch.cuda.is_available()
    device=torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model=model.cuda()

    total_acc_test=0
    with torch.no_grad():
        count=0
        predictions=[]
        for test_input,test_label in tqdm(test_dataloader):
            count+=batchsize
            test_label=test_label.to(device)
            mask=test_input['attention_mask'].to(device)
            input_id=test_input['input_ids'].squeeze(1).to(device)
            output=model(input_id,mask)
            #print(output.argmax(dim=1),test_label)
            predictions.append(output.argmax(dim=1))  #save the prediction for further analysis
            acc=(output.argmax(dim=1)==test_label).sum().item()

            total_acc_test+=acc
            if count%100==0:
                print(f'Accuracy so far = {total_acc_test/count: .3f}')

    print(f'Test accuracy: {total_acc_test/len(test_dataset): .3f}')
    return predictions

In [ ]:
predictions=evaluate(model, prop_test_data)

In [ ]:
flattened=[]
for batch in predictions:
    for pred in batch:
        flattened.append(reverse_index[pred.item()])
propaganda_val_set['prediction']=flattened
propaganda_val_set.head(10)

In [ ]:
all_labels=propaganda_val_set['label']
all_predictions=propaganda_val_set['prediction']

In [ ]:
tp={}
fp={}
fn={}
tn={}

for label1,pred1 in zip(all_labels,all_predictions):
    for label in prop_labels.keys():
        if label1==label:
            if pred1==label:
                tp[label]=tp.get(label,0)+1
            else:
                fn[label]=fn.get(label,0)+1

        else:
            if pred1==label:
                fp[label]=fp.get(label,0)+1
            else:
                tn[label]=tn.get(label,0)+1



precision={label:value/(value+fp.get(label,0)) for label,value in tp.items()}
recall={label:value/(value+fn.get(label,0)) for label,value in tp.items()}
f1={label:(2*value*recall.get(label,0))/(value+recall.get(label,0)) for label,value in precision.items()}


In [ ]:
recall

In [ ]:
precision

In [ ]:
f1